In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.utils import shuffle
from sklearn import preprocessing

songs=pd.read_csv('data/tracks.csv')
# songs = shuffle(songs)
songs=songs.head(25000)

In [ ]:
songs_for_svr = songs.copy()
songs_for_svr.set_index('id',inplace=True)

In [ ]:
def drop_columns(dataframe,to_be_deleted):
    dataframe.drop(to_be_deleted, axis=1, inplace=True)
to_be_deleted = ['id_artists', 'artists','name','release_date'] 

In [ ]:
songs_for_svr['release_date'] = pd.to_datetime(songs_for_svr['release_date'], errors='coerce')

In [ ]:
songs_for_svr['year'] = pd.DatetimeIndex(songs_for_svr['release_date']).year
songs_for_svr['month'] = pd.DatetimeIndex(songs_for_svr['release_date']).month

In [ ]:
drop_columns(songs_for_svr,to_be_deleted)#delete columns

In [ ]:
def scale_whole_df_except_valence():
    valence_only = songs_for_svr['valence']
    del songs_for_svr["valence"]

    x = songs_for_svr #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    songs_for_svr[songs_for_svr.columns] = min_max_scaler.fit_transform(songs_for_svr[songs_for_svr.columns].to_numpy())
    songs_for_svr['valence'] = valence_only
scale_whole_df_except_valence()

In [ ]:
songs_for_svr["key"] = songs_for_svr["key"].astype("category")
songs_for_svr = pd.get_dummies(songs_for_svr, columns=["key"])

In [ ]:
starting_key = 0 
for i in range (16, 28):
    songs_for_svr = songs_for_svr.rename(columns={songs_for_svr.columns[i]: 'key_%d' % starting_key})
    starting_key = starting_key  + 1 

In [ ]:
songs_for_svr.dropna(inplace=True)

In [ ]:
x = songs_for_svr['valence']
del songs_for_svr["valence"]
songs_for_svr['valence'] = x

# AND SO IT BEGINS

In [ ]:
independent_variables = songs_for_svr.columns.drop('valence')
y = songs_for_svr['valence']
X = songs_for_svr[independent_variables]

In [ ]:

from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(
                        X, np.ravel(y),
                test_size = 0.20, random_state = 101)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from sklearn.svm import SVR
model = SVR()
model.fit(X_train, y_train)
 
# print prediction results
predictions = model.predict(X_test)

In [ ]:
from sklearn import metrics 
metrics.mean_absolute_error(y_test, predictions)

In [ ]:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3, cv =3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

BEST PARAMS: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
metrics.mean_absolute_error(y_test, grid_predictions)

0.1224 me c10 gamma 0.01 kai sample 25000

In [ ]:
from sklearn.inspection import permutation_importance
perm_importance = permutation_importance(grid, X_test, y_test)

In [ ]:
perm_importance


In [ ]:
import matplotlib.pyplot as plt

feature_names = X_test.columns
features = np.array(feature_names)

sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

11.53